In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from bt_classes import my_backtest, test_indicator
# orig_df = pd.read_csv('../res/test_data.csv',index_col=0,parse_dates=True)
# my_backtest(orig_df[:500])
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
rcParams['figure.figsize'] = 20,10
# df = pd.read_csv('../res/input0130.csv')

orig_df = pd.read_csv('../xau_1d_20y.csv')
orig_df['datetime'] = pd.to_datetime(orig_df['date'])
orig_df = orig_df.set_index('datetime')

df = orig_df.copy()
df['log_r'] = np.log(df['close']) - np.log(df['open'])
df['label'] = np.sign(df['log_r'].shift(-1))
df['label'][df['label']==-1] = 0
df['label'].fillna(0,inplace=True)

In [2]:
#importing required libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
# follow the literature
# we don't use min-max scaling here, use partial mean-std scaling instead
from sklearn.preprocessing import StandardScaler
from itertools import chain

# and we define our model here
def lstm_model(sample_len=240,para_a=42, para_b=17):
    model = Sequential()
    model.add(LSTM(units=para_a, dropout=0.1, return_sequences=True, input_shape=(sample_len,1),activation='tanh'))# (25,15)-57, (42,17)-58
    model.add(LSTM(units=para_b, dropout=0.08, activation='tanh'))
    # model.add(Dropout(0.08))# 加了之后同原先效果差不多，（应该一定程度上）可以防止过拟合
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [6]:
sample_len = 15
train_begin = sample_len
train_end = train_begin + 2000
scaler = StandardScaler()
train_set = df[['log_r','label']][train_begin-sample_len:train_end].reset_index()
x_train, y_train = [], []
x_train_set = list(chain.from_iterable(scaler.fit_transform(train_set['log_r'].values.reshape(-1,1))))
for i in range(sample_len,len(x_train_set)):
    x_train.append(x_train_set[i-sample_len:i])
    y_train.append(train_set['label'][i])
x_train, y_train = np.array(x_train), np.array(y_train)
y_train = to_categorical(y_train,2)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1)) 

model = lstm_model(sample_len=sample_len)
model.fit(x_train,y_train,epochs=100, batch_size=100, callbacks=[EarlyStopping(monitor='loss',patience=10)])

Epoch 1/100
2000/2000 [==============================] - 1s 507us/step - loss: 0.6930 - accuracy: 0.5075
Epoch 2/100
2000/2000 [==============================] - 1s 295us/step - loss: 0.6921 - accuracy: 0.5135
Epoch 3/100
2000/2000 [==============================] - 1s 297us/step - loss: 0.6917 - accuracy: 0.5220
Epoch 4/100
2000/2000 [==============================] - 1s 296us/step - loss: 0.6922 - accuracy: 0.5130
Epoch 5/100
2000/2000 [==============================] - 1s 290us/step - loss: 0.6913 - accuracy: 0.5190
Epoch 6/100
2000/2000 [==============================] - 1s 291us/step - loss: 0.6921 - accuracy: 0.5175
Epoch 7/100
2000/2000 [==============================] - 1s 292us/step - loss: 0.6924 - accuracy: 0.5195
Epoch 8/100
2000/2000 [==============================] - 1s 291us/step - loss: 0.6909 - accuracy: 0.5165
Epoch 9/100
2000/2000 [==============================] - 1s 299us/step - loss: 0.6912 - accuracy: 0.5265
Epoch 10/100
2000/2000 [==============================]

In [7]:
model.save(f'd{sample_len}-{sample_len+2000}.h5')  # d日频，240-2240天